In [ ]:
from typing import Optional
import requests
from bs4 import BeautifulSoup
from smolagents import CodeAgent, LiteLLMModel, FinalAnswerTool, ToolCallingAgent, tool, Tool

In [ ]:
# Tracing 

# from phoenix.otel import register
# from openinference.instrumentation.smolagents import SmolagentsInstrumentor

# register()
# SmolagentsInstrumentor().instrument()

In [30]:
@tool

def get_holidays(day: int, month: int) -> str:
    """A tool that returns holidays for a given day and month.
    
    Args:
        day: The day of the month (1-31) in integer format
        month: The month of the year (1-12) in integer format
    
    Returns:
        A string listing holidays for the specified date, or an error message if something goes wrong.
    """
    #Imports
    
    import requests
    from bs4 import BeautifulSoup
    
    #Mapping for request 
    
    months = {
    1:"yanvar",
    2:"fevral",
    3:"mart",
    4:"aprel",
    5:"may",
    6:"iyun",
    7:"iyul",
    8:"avgust",
    9:"sentyabr",
    10:"oktyabr",
    11:"noyabr",
    12:"dekabr"
    }
    
    # Validate the month and day
    if month is None or day is None:
        return "Invalid month/day"

    # Validate month range
    if not (1 <= month <= 12):
        return "Invalid month"

    # Validate day range (assuming non-leap year for simplicity)
    if month in {4, 6, 9, 11} and day > 30:
        return "Invalid day for this month"
    elif month == 2 and day > 28:
        return "Invalid day for February"
    elif day > 31:
        return "Invalid day"

    # Map the month to its Russian name
    russian_month = months[month]

    # Construct the query URL
    url = f"https://kakoysegodnyaprazdnik.ru/baza/{russian_month}/{day}"

    # Define headers to mimic a browser
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows; Windows NT 10.3; WOW64; en-US) AppleWebKit/603.45 (KHTML, like Gecko) Chrome/51.0.2756.132 Safari/601", 
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "ru-RU,en;q=0.9",
        "Accept-Encoding": "gzip, deflate"
    }

    try:
        # Send GET request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an exception for 4xx/5xx errors
        response.encoding = 'utf-8'  # Ensure proper decoding of Russian text

        # Parse HTML content
        soup = BeautifulSoup(response.text, "html.parser")
        listing_wr_div = soup.find("div", class_="listing_wr")

        if not listing_wr_div:
            return "No holiday data found for this date."

        # Extract holiday names
        holidays = []
        for answer_div in listing_wr_div.find_all("div", itemprop=["suggestedAnswer", "acceptedAnswer"]):
            span_tag = answer_div.find("span", itemprop="text")
            if span_tag:
                holiday = span_tag.get_text(strip=True)
                holidays.append(holiday)

        # Return results
        if not holidays:
            return "No holidays found for this date."
        return "\n".join(holidays)  # Join holidays with newlines for readability

    except requests.RequestException as e:
        return f"Failed to retrieve holidays. Error: {str(e)}"


In [27]:


model = LiteLLMModel(
    model_id="ollama/phi4:latest",
    api_base="http://localhost:8888",
    api_key="YOUR_API_KEY", 
    num_ctx=8192
)

agent = ToolCallingAgent(tools=[get_holidays], model=model)

agent.run("Какие есть кринжовые праздники на 24 марта? Ответь на русском в формате таблицы Markdown - Праздник, Тема, Уровень кринжа (1-10)")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Какие есть кринжовые праздники на 24 марта? Ответь на русском в формате таблицы Markdown - Праздник, Тема,      │
│ Уровень кринжа (1-10)                                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/phi4:latest ─────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'get_holidays' with arguments: {'day': 24, 'month': 3}                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: День мастера маникюра
День рождения паровоза
День пробуждения медведя
Международный день борьбы с депрессией (International Day Against Depression)
День вверх тормашками
День врача-фтизиатра
День штурманской службы ВВС РФ
День рождения куриного окорочка
Международный день иллюзионистов
Всемирный день борьбы с туберкулёзом (World Tuberculosis Day)
Международный день достижений (International Day for Achievers)
Международный день права на установление истины в отношении грубых нарушений прав человека и достоинства жертв 
(International Day for the Right to the Truth Concerning Gross Human Rights Violations and for the Dignity of 
Victims)
День сноса телебашни
День выключения (Shutdown Day)
Международный день осведомителя (International Whistleblower Day)
Всемирный день кабадди (World Kabaddi Day)
День памяти священномученика Пиония Смирнского
День памяти святителя Евфимия Новгородского
День памяти преподобного Алексия Голосеевского
День памяти святителя Софрония Иерусалимского
День памяти жертв агрессии НАТО против Югославии (Remembrance Day for Victims of NATO Aggression) - Сербия
День коктейлей (National Cocktail Day) - США
День посадки деревьев - Уганда
День жизни (Narodowy Dzien Zycia) - Польша
День изюма в шоколаде (National Chocolate Covered Raisin Day) - США
День соседей (Flatmate`s Day) - США
День чизстейка (National Cheesesteak Day) - США
День соглашения (Commonwealth Covenant Day) - Северные Марианские острова
День памяти ради истины и справедливости (Dia de la Memoria por la Verdad y la Justicia) - Аргентина
День Отаго (Otago Anniversary Day) - Новая Зеландия
День святого Иосифа (Saint Joseph`s Day) - Колумбия
Именины у Ивана, Георгия, Василия, Ефима, Софрония, Пиония, Епимаха
Ефимов день (Антип-зубник)

[Step 0: Duration 1.24 seconds| Input tokens: 1,189 | Output tokens: 34]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '| Праздник | Тема | Уровень кринжа                     │
│ |\n|---------------------|---------------|-------------------|\n| День мастера маникюра | Косметология и забота │
│ о ногтях | 3 |\n| День рождения паровоза | Паровозы и железные дороги | 4 |\n| День пробуждения медведя |       │
│ Медведи и природа | 2 |\n| Международный день борьбы с депрессией | Здравоохранение и психология | 1 |\n| День  │
│ вверх тормашками | Юмористический подход к повседневной жизни | 9 |\n| День штурманской службы ВВС РФ | Военные │
│ и авиация | 3 |\n| День рождения куриного окорочка | Кулинария и продукты питания | 5 |\n| Международный день   │
│ иллюзионистов | Искусство и развлечения | 6 |\n| Всемирный день борьбы с туберкулёзом | Здравоохранение | 1     │
│ |\n| Международный день права на установление истины | Права человека | 2 |\n| День сноса телебашни |           │
│ Архитектура и инженерия | 7 |\n| День выключения (Shutdown Day) | Экология и технологии | 4 |\n| Международный  │
│ день осведомителя | Права человека и этика | 2 |\n| Всемирный день кабадди | Спорт | 3 |\n| День памяти         │
│ святителя Евфимия Новгородского | Религия и традиции | 1 |\n| День памяти жертв агрессии НАТО против Югославии  │
│ (Сербия) | Исторические события | 4 |\n| День коктейлей (США) | Культура и напитки | 5 |\n| День посадки        │
│ деревьев (Уганда) | Экология и природа | 1 |\n| Национальный день чизстейка (США) | Кулинария | 7 |\n| День     │
│ соглашения (Северные Марианские острова) | Исторические события и политика | 6 |\n| День памяти ради истины и   │
│ справедливости (Аргентина) | Право и общество | 3 |'}                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: | Праздник | Тема | Уровень кринжа |
|---------------------|---------------|-------------------|
| День мастера маникюра | Косметология и забота о ногтях | 3 |
| День рождения паровоза | Паровозы и железные дороги | 4 |
| День пробуждения медведя | Медведи и природа | 2 |
| Международный день борьбы с депрессией | Здравоохранение и психология | 1 |
| День вверх тормашками | Юмористический подход к повседневной жизни | 9 |
| День штурманской службы ВВС РФ | Военные и авиация | 3 |
| День рождения куриного окорочка | Кулинария и продукты питания | 5 |
| Международный день иллюзионистов | Искусство и развлечения | 6 |
| Всемирный день борьбы с туберкулёзом | Здравоохранение | 1 |
| Международный день права на установление истины | Права человека | 2 |
| День сноса телебашни | Архитектура и инженерия | 7 |
| День выключения (Shutdown Day) | Экология и технологии | 4 |
| Международный день осведомителя | Права человека и этика | 2 |
| Всемирный день кабадди | Спорт | 3 |
| День памяти святителя Евфимия Новгородского | Религия и традиции | 1 |
| День памяти жертв агрессии НАТО против Югославии (Сербия) | Исторические события | 4 |
| День коктейлей (США) | Культура и напитки | 5 |
| День посадки деревьев (Уганда) | Экология и природа | 1 |
| Национальный день чизстейка (США) | Кулинария | 7 |
| День соглашения (Северные Марианские острова) | Исторические события и политика | 6 |
| День памяти ради истины и справедливости (Аргентина) | Право и общество | 3 |

[Step 1: Duration 20.97 seconds| Input tokens: 3,239 | Output tokens: 811]

'| Праздник | Тема | Уровень кринжа |\n|---------------------|---------------|-------------------|\n| День мастера маникюра | Косметология и забота о ногтях | 3 |\n| День рождения паровоза | Паровозы и железные дороги | 4 |\n| День пробуждения медведя | Медведи и природа | 2 |\n| Международный день борьбы с депрессией | Здравоохранение и психология | 1 |\n| День вверх тормашками | Юмористический подход к повседневной жизни | 9 |\n| День штурманской службы ВВС РФ | Военные и авиация | 3 |\n| День рождения куриного окорочка | Кулинария и продукты питания | 5 |\n| Международный день иллюзионистов | Искусство и развлечения | 6 |\n| Всемирный день борьбы с туберкулёзом | Здравоохранение | 1 |\n| Международный день права на установление истины | Права человека | 2 |\n| День сноса телебашни | Архитектура и инженерия | 7 |\n| День выключения (Shutdown Day) | Экология и технологии | 4 |\n| Международный день осведомителя | Права человека и этика | 2 |\n| Всемирный день кабадди | Спорт | 3 |\n| 